# FEC Fed. Campaign Contributions

Download a set of bulk data files from:
http://classic.fec.gov/finance/disclosure/ftpdet.shtml

Put them in a directory and unzip them to create the readable text data files.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
my_dir = './FEC/2018/'
indiv_header_file = my_dir + 'indiv_header_file.csv'
indiv_file = my_dir + 'itcont.txt' # individual donations

ccl_header_file = my_dir + 'ccl_header_file.csv'
ccl_file = my_dir + 'ccl.txt'  # candidate-to-committee linkage

cn_header_file = my_dir + 'cn_header_file.csv'
cn_file = my_dir + 'cn.txt'  # Federal candidates

cm_header_file = my_dir + 'cm_header_file.csv'
cm_file = my_dir + 'cm.txt'  # Committees (funds and PACs)

In [ ]:
def get_columns(header_file) -> list:
    with open(header_file, 'r') as hf:
        top_line = hf.readline().strip()
    return top_line.split(',')

In [ ]:
# Read the Candidates file header:
cn_cols = get_columns(cn_header_file)

In [ ]:
cn_cols

In [ ]:
# Read the Candidates file data:
cn = pd.read_csv(cn_file, sep='|', index_col=0, names=cn_cols)

In [ ]:
cn.head()

## Minor cleanup needed, for NaN values and floats -> ints...

In [ ]:
# Read the Committee header:
cm_cols = get_columns(cm_header_file)
# Read the Committee file data:
cm = pd.read_csv(cm_file, sep='|', index_col=0, names=cm_cols)

In [ ]:
cm.head()

In [ ]:
# Select all committees that are Corporations:  ('C' or 'W')
c = cm['ORG_TP'] == 'C'
w = cm['ORG_TP'] == 'W'

cm[c | w]

In [ ]:
# Read the Individuals header:
indiv_cols = get_columns(indiv_header_file)
# Read the Committee file data:
#  NOTE: The unique ID is the LAST column, not the first

my_date_parser = lambda dates: [pd.datetime.strptime(d, '%d%m%Y') for d in dates]

indiv = pd.read_csv(indiv_file, sep='|', index_col='SUB_ID', 
                    names=indiv_cols,
                    usecols=['NAME','CITY','STATE','ZIP_CODE',
                            'EMPLOYER','OCCUPATION',
                            'TRANSACTION_DT','TRANSACTION_AMT',
                            'CMTE_ID','ENTITY_TP','SUB_ID'],
                    )

In [ ]:
indiv.tail()

In [ ]:
# How much memory (in bytes) does this huge DataFrame require?
indiv.memory_usage(index=True).sum()

# Some simple queries:

### Find individual contributions from Champaign, IL:

In [ ]:
champaign = indiv[(indiv['CITY'] == 'CHAMPAIGN') & (indiv['STATE'] == 'IL')]

In [ ]:
champaign.count()

In [ ]:
# Find all the unique donor names from Champaign, and alphabetize them:
sorted(champaign['NAME'].unique())

In [ ]:
champaign.head()

In [ ]:
champaign.memory_usage(index=True).sum()

In [ ]:
indiv.memory_usage(index=True).sum()

## Let's expand a bit, to get all donations from Champaign, Urbana, and Savoy together:

In [ ]:
local = indiv[ ((indiv['CITY'] == 'CHAMPAIGN') |
                (indiv['CITY'] == 'URBANA') |
                (indiv['CITY'] == 'SAVOY') )
             & (indiv['STATE'] == 'IL')]

In [ ]:
# How many donations from EACH city?  Uses groupby()
local.groupby(['CITY']).count()

In [ ]:
# Total and average donations from EACH city?
print(local.groupby(['CITY'])['TRANSACTION_AMT'].sum())
print(local.groupby(['CITY'])['TRANSACTION_AMT'].mean())

# Joining Related DataFrames 
For example, if we want to know to which these individual donations went, we must join the Committee file.

In [ ]:
indiv.head(2)

In [ ]:
cm[(cm.index == 'C00629618') | (cm.index == 'C00177436')]

In [ ]:
# using the isin() method to match multiple rows without the messy booleans above
cm[cm.index.isin(['C00629618','C00177436'])]

In [ ]:
# We can join these using the merge() function:
pd.merge(indiv, cm, right_index=True, left_on='CMTE_ID').head()

In [ ]:
# Or the join() method
indiv.join(cm, on='CMTE_ID').head()

In [ ]:
champaign_plus_cm = champaign.join(cm, on='CMTE_ID')

In [ ]:
champaign_plus_cm[['NAME', 'EMPLOYER', 'OCCUPATION', 'TRANSACTION_AMT', 'CMTE_NM', 'CONNECTED_ORG_NM', 'CMTE_PTY_AFFILIATION']]

In [ ]:
uiuc = champaign_plus_cm[champaign_plus_cm['EMPLOYER'] == 'UNIVERSITY OF ILLINOIS']
uiuc[['NAME', 'EMPLOYER', 'OCCUPATION', 'TRANSACTION_AMT', 'CMTE_NM', 'CONNECTED_ORG_NM', 'CMTE_PTY_AFFILIATION']]

# Things to do:
* Determine total individual donations PER STATE.
* List the 100 individuals nationwide who gave the most total dollars (not # of transactions) and what the total was.
* List Senators & Representatives and which Corporations and other PACs have "bought" them with huge donations.
* 
* 